In [24]:
import pandas as pd
import json
import numpy as np

# Transformação dos dados TJ - Educação

## Carregar os dados

In [25]:
years = [year for year in range(2010, 2024)]

df = pd.DataFrame()
for year in years:
    # Lê o arquivo csv correspondente ao ano
    aux = pd.read_csv(f'../data/{year}.csv', encoding='latin-1', sep=';')
    aux['ANO'] = year

    df = pd.concat([df, aux], ignore_index=True,).reset_index(drop=True)

df.to_csv("../data/data.csv", index=False)

df.head()

,NO_MUNICIPIO,TP_DEPENDENCIA,TP_CATEGORIA_ESCOLA_PRIVADA,TP_LOCALIZACAO,TP_LOCALIZACAO_DIFERENCIADA,TP_REGULAMENTACAO,TP_OCUPACAO_PREDIO_ESCOLAR,IN_PREDIO_COMPARTILHADO,IN_AGUA_FILTRADA,IN_AGUA_POTAVEL,...,Unnamed: 128,Unnamed: 129,IN_PROF_GESTAO,IN_ACESSIBILIDADE_ELEVADOR,IN_PROF_COORDENADOR,QT_PROF_COORDENADOR,IN_ALMOXARIFADO,IN_AREA_VERDE,IN_AUDITORIO,QT_MAT_FUND_INT
0,Ilhéus,2.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Ilhéus,3.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ilhéus,3.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Ilhéus,3.0,0.0,1.0,0.0,2.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Ilhéus,4.0,4.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Processamento nos dados do DF

In [27]:
# Função para converter float para int
def float_to_int(value):
    try:
        return int(value)
    except:
        return "NaN"

# Converte as colunas para inteiros
for column in df.columns:
    if column != 'NO_MUNICIPIO':
        df[column] = df[column].apply(float_to_int)

    if 'Unnamed' in column:
        del df[column]

columns = df.columns
columns = [column for column in columns if column not in ['NO_MUNICIPIO', 'ANO']]
df = df[['NO_MUNICIPIO', 'ANO'] + columns]

# Preenche os valores NaN
df.fillna('NaN', inplace=True)

# Carrega o dicionário de categorias
dict_tp = json.load(open('../data/dict_categoria.json', 'r'))

columns = df.columns
columns = ['NO_MUNICIPIO', 'ANO'] + [column for column in columns if column in dict_tp.keys()]
df = df[columns]

# Loop para mapear as categorias
for key in dict_tp:
    # Verifica se a coluna existe no dataframe
    if key in df.columns:
        tmp = {}

        # Mapeia as categorias
        if 'categoria' in dict_tp[key]:
            for k, v in dict_tp[key]['categoria'].items():
                # Verifica se o valor é NaN
                if k == 'NaN':
                    tmp[k] = v

                    if '0' not in tmp.keys() and 0 not in tmp.keys():
                        tmp[0] = v
                else:
                    tmp['NaN'] = "Não aplicável"
                    tmp[int(k)] = v

            df[key] = df[key].map(tmp)

for column in df.columns:
    if column not in dict_tp.keys() and 'IN' in column:
        df[column] = df[column].map({0: 'Não', 1: 'Sim', 'NaN': 'Não aplicável'})

df.replace('NaN', "Não aplicável", inplace=True)

df.head()

,NO_MUNICIPIO,ANO,TP_DEPENDENCIA,TP_CATEGORIA_ESCOLA_PRIVADA,TP_LOCALIZACAO,TP_LOCALIZACAO_DIFERENCIADA,TP_REGULAMENTACAO,TP_OCUPACAO_PREDIO_ESCOLAR,IN_PREDIO_COMPARTILHADO,IN_AGUA_FILTRADA,...,QT_MAT_INF_PRE_INT,QT_DOC_INF,QT_DOC_INF_CRE,QT_DOC_INF_PRE,QT_TUR_INF,QT_TUR_INF_CRE,QT_TUR_INF_PRE,IN_PROF_GESTAO,IN_PROF_COORDENADOR,QT_PROF_COORDENADOR
0,Ilhéus,2010,Estadual,Não aplicável para escolas públicas,Urbana,Escola não está em área de localização diferen...,Sim,Não aplicável para escolas que não ocupam préd...,Não,Sim,...,0,0,0,0,0,0,0,Não aplicável,Não aplicável,Não aplicável
1,Ilhéus,2010,Municipal,Não aplicável para escolas públicas,Urbana,Escola não está em área de localização diferen...,Sim,Não aplicável para escolas que não ocupam préd...,Não,Sim,...,0,0,0,0,0,0,0,Não aplicável,Não aplicável,Não aplicável
2,Ilhéus,2010,Municipal,Não aplicável para escolas públicas,Urbana,Escola não está em área de localização diferen...,Sim,Não aplicável para escolas que não ocupam préd...,Não,Sim,...,39,7,3,4,6,2,4,Não aplicável,Não aplicável,Não aplicável
3,Ilhéus,2010,Municipal,Não aplicável para escolas públicas,Urbana,Escola não está em área de localização diferen...,Em tramitação,Não aplicável para escolas que não ocupam préd...,Não,Sim,...,0,4,0,4,3,0,3,Não aplicável,Não aplicável,Não aplicável
4,Ilhéus,2010,Privada,Filantrópica,Urbana,Escola não está em área de localização diferen...,Sim,Não aplicável para escolas que não ocupam préd...,Não,Sim,...,0,3,2,1,3,2,1,Não aplicável,Não aplicável,Não aplicável


In [28]:
try:
    with open('../data/cities.txt', 'r') as fd:
        cities = fd.readlines()
        cities = [city.strip().replace("\n", "") for city in cities]
    cities = ['Itabuna', 'Ilhéus']

    print("Before", df.shape)
    df = df[df['NO_MUNICIPIO'].isin(cities)]
    print("After", df.shape)
except FileNotFoundError:
    print("Arquivo não encontrado")


Before (32571, 110)
After (4227, 110)


In [29]:
df.to_csv("../output/data.csv", index=False)
df.head()
df[df['NO_MUNICIPIO'] == 'Ilhéus']

,NO_MUNICIPIO,ANO,TP_DEPENDENCIA,TP_CATEGORIA_ESCOLA_PRIVADA,TP_LOCALIZACAO,TP_LOCALIZACAO_DIFERENCIADA,TP_REGULAMENTACAO,TP_OCUPACAO_PREDIO_ESCOLAR,IN_PREDIO_COMPARTILHADO,IN_AGUA_FILTRADA,...,QT_MAT_INF_PRE_INT,QT_DOC_INF,QT_DOC_INF_CRE,QT_DOC_INF_PRE,QT_TUR_INF,QT_TUR_INF_CRE,QT_TUR_INF_PRE,IN_PROF_GESTAO,IN_PROF_COORDENADOR,QT_PROF_COORDENADOR
0,Ilhéus,2010,Estadual,Não aplicável para escolas públicas,Urbana,Escola não está em área de localização diferen...,Sim,Não aplicável para escolas que não ocupam préd...,Não,Sim,...,0,0,0,0,0,0,0,Não aplicável,Não aplicável,Não aplicável
1,Ilhéus,2010,Municipal,Não aplicável para escolas públicas,Urbana,Escola não está em área de localização diferen...,Sim,Não aplicável para escolas que não ocupam préd...,Não,Sim,...,0,0,0,0,0,0,0,Não aplicável,Não aplicável,Não aplicável
2,Ilhéus,2010,Municipal,Não aplicável para escolas públicas,Urbana,Escola não está em área de localização diferen...,Sim,Não aplicável para escolas que não ocupam préd...,Não,Sim,...,39,7,3,4,6,2,4,Não aplicável,Não aplicável,Não aplicável
3,Ilhéus,2010,Municipal,Não aplicável para escolas públicas,Urbana,Escola não está em área de localização diferen...,Em tramitação,Não aplicável para escolas que não ocupam préd...,Não,Sim,...,0,4,0,4,3,0,3,Não aplicável,Não aplicável,Não aplicável
4,Ilhéus,2010,Privada,Filantrópica,Urbana,Escola não está em área de localização diferen...,Sim,Não aplicável para escolas que não ocupam préd...,Não,Sim,...,0,3,2,1,3,2,1,Não aplicável,Não aplicável,Não aplicável
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32536,Ilhéus,2023,Estadual,Não aplicável para escolas públicas,Urbana,Escola não está em área de localização diferen...,Sim,Próprio,Não,Não aplicável,...,0,0,0,0,0,0,0,Sim,Não aplicável,Não aplicável
32537,Ilhéus,2023,Municipal,Não aplicável para escolas públicas,Urbana,Escola não está em área de localização diferen...,Sim,Alugado,Não,Não aplicável,...,0,0,0,0,0,0,0,Sim,Não aplicável,Não aplicável
32538,Ilhéus,2023,Privada,Filantrópica,Urbana,Escola não está em área de localização diferen...,Sim,Cedido,Não,Não aplicável,...,0,4,1,3,3,1,2,Não,Não aplicável,Não aplicável
32555,Ilhéus,2023,Privada,Particular,Urbana,Escola não está em área de localização diferen...,Sim,Alugado,Não,Não aplicável,...,0,0,0,0,0,0,0,Sim,Não aplicável,Não aplicável


## Agrupar e converter dados para JSON

In [30]:
""" tmp = df[['ANO']].copy()
tmp =  tmp.rename(columns={'ANO': 'count'})
df = pd.concat([df, tmp], axis=1)

out_json = {}

columns = df.columns
columns = [column for column in columns if column not in ['NO_MUNICIPIO', 'ANO', 'count']]

for column in columns:
    groupby_df = df.groupby(['NO_MUNICIPIO', 'ANO', column],
                            as_index=False).agg({'count': 'count'})
    tmp = groupby_df.to_dict(orient='split')

    for i in range(len(tmp['data'])):
        municipio = tmp['data'][i][0]
        ano = tmp['data'][i][1]
        categoria = tmp['data'][i][2]
        count = tmp['data'][i][3]

        if municipio not in out_json:
            out_json[municipio] = {}

        if ano not in out_json[municipio]:
            out_json[municipio][ano] = {}

        if column not in out_json[municipio][ano]:
            out_json[municipio][ano][column] = {}

        out_json[municipio][ano][column][categoria] = count

json.dump(out_json, open('../output/data.json', 'w'), ensure_ascii=False) """

" tmp = df[['ANO']].copy()\ntmp =  tmp.rename(columns={'ANO': 'count'})\ndf = pd.concat([df, tmp], axis=1)\n\nout_json = {}\n\ncolumns = df.columns\ncolumns = [column for column in columns if column not in ['NO_MUNICIPIO', 'ANO', 'count']]\n\nfor column in columns:\n    groupby_df = df.groupby(['NO_MUNICIPIO', 'ANO', column],\n                            as_index=False).agg({'count': 'count'})\n    tmp = groupby_df.to_dict(orient='split')\n\n    for i in range(len(tmp['data'])):\n        municipio = tmp['data'][i][0]\n        ano = tmp['data'][i][1]\n        categoria = tmp['data'][i][2]\n        count = tmp['data'][i][3]\n\n        if municipio not in out_json:\n            out_json[municipio] = {}\n\n        if ano not in out_json[municipio]:\n            out_json[municipio][ano] = {}\n\n        if column not in out_json[municipio][ano]:\n            out_json[municipio][ano][column] = {}\n\n        out_json[municipio][ano][column][categoria] = count\n\njson.dump(out_json, open('../ou